# 1 Traditional Machine Learning on Graphs

In a traditional ML pipeline such as Logistic regression, Random Forest, and Neural
networks, the model is first trained on features of a graph and then the model can be
applied on a new graph. Using effective features over graphs is the key to achieving good
model performance. For simplicity, in this section we focus on un-directed graphs.

## 1.1 Node-Level Features

A simple example of node-level task is node classification based on a few samples. It is
illustrated in Figure 1.2

A couple different measures characterise the structure and position of a node on a graph:

**Node degree:** Number of neighbours

**Node centralities:** A measure of the “importance” of a node in a graph. There
several different types of centrality.
- **Eigenvector centrality:** A node is important if it is surrounded by important
neighbouring nodes. We define the centrality of node v as the centrality of
neighbouring nodes. This leads to a set of $|N|$ simultaneous linear equations:
$$
c_v := \frac{1}{\lambda} \sum_{u \in N(v)} c_u \quad \Longleftrightarrow \quad \lambda c = Ac
$$
where $λ$ is a normalisation constant and $A$ is the adjacency matrix of the graph.
By Perron-Forbenius Theorem, the largest eigenvalue $\lambda_{\text{max}}$ is always positive
and unique, and its corresponding eigenvectors can be used for centrality.
When $λ$ is the second-largest eigenvalue, $c_v$ has a different meaning.

![f4](f4.jpeg)

> Figure 1.3: Examples of clustering coefficients

- **Betweenness centrality:** A node is important if it is a gatekeeper. i.e. when
it lies on many shortest paths between other nodes.

$$
c_j = \sum_{s \neq t \neq j} \frac{|\text{{shortest paths between }} s \text{{ and }} t \text{{ containing }} j|}{|\text{{shortest paths between }} s \text{{ and }} t|}
$$
This is important for social networks.

- **Closeness Centrality**:
  $$
  c_v := \frac{1}{\sum_{u \neq v} [\text{{shortest paths between }} u \text{{ and }} v]}
  $$

- **Clustering Coefficients:** How connected \( v \)'s neighbouring nodes are:
  $$
  e_v := \frac{1}{{\binom{k_v}{2}}} |\{\text{{edges among }} N(v)\}| \in [0, 1]
  $$
Social networks have a huge amount of clustering.

Clustering co¨efficient counts the number of triangles in the **ego-network** (the network
formed by ${v} ∪ N(v)$, where $v$ is the ego). We can generalise the above by counting
graphlets.
An **induced graph** is a graph formed by taking a subset of vertices in a larger graph such
that only edges connecting the remaining vertices are preserved.
Two graphs with identical topologies are **isomorphic**.

![f5](f5.jpeg)

> Figure 1.4: Different levels of features distinguish nodes in different ways

**Graphlets** are small subgraphs that describe the structure of u’s neighbourhood network.
Specifically, they are *rooted, connected, induced, non-isomorphic* subgraphs. Considering
graphs of size 2 to 5 nodes we get a vector of 73 (number of graphlets with 2 to 5 vertices)
elements that describes the topology of a node’s neighbourhood. This vector is the
**graphlet degree vector (GDV)** of a node.

The features we have discussed so far capture local topological properties of the graph but
cannot distinguish points in a global scale.

## 1.2 Link-Level Features

Two formulations of link-level prediction
- Links missing at random: Remove a random set of links and then aim to predict
them.</br>
    e.g. drug interactions
- Links over time: Given $G[t_0, t'_0]$ a graph defined by edges up to time $t'0$ output a
rankd list L of edges that are predicted to appear in time $G[t_1, t'_1]$. </br>
    Evaluation: $n = |Enew|$: number of new edges that appear during the test period.
Methodology: For each pair of nodes $x, y$ compute a distance $c(x, y)$ and predict top
n elements as links.

A couple measures exist for local neighbourhood overlap:

- **Common neighbours**:
  $$
  c(u, v) := |N(v_1) \cap N(v_2)|
  $$

- **Jaccard’s Coefficient**:
  $$
  c(u, v) := \frac{|N(v_1) \cap N(v_2)|}{|N(v_1) \cup N(v_2)|}
  $$

- **Adamic-Adar Index**:
  $$
  c(u, v) := \sum_{u \in N(v_1) \cap N(v_2)} \frac{1}{k_u}
  $$
  The problem with the three indices above is that they are always 0 if $u, v$ do not share a neighbour.

- **Katz Index**:
  $$
  c(u, v) := \text{[All walks of all lengths between } u \text{ and } v\text{]}
  $$
  This can be computed by powers of the adjacency matrix. The matrix counting all walks of length \( n \) between vertices is $A^n$, so the Katz index can be computed by:
  $$
  C := \sum_{i=1}^{\infty} \beta^i A^i = (I - \beta A)^{-1} - I
  $$
  where the $\beta < 1$ decay factor is necessary to prevent **$C$** from blowing up to  $+\infty$. An analogous definition exists for directed graphs.



## 1.3 Graph Kernels

The goal of graph kernels is to create a feature vector for the entire graph.

**Kernel methods** are widely used for traditional ML for graph-level prediction. Instead of
designing feature vectors, we design kernels:
- $k(G,G′) ∈ R$
- Kernel matrix **$K$** $= [K(G,G')]_{G,G'}$ must always be positive semi-definite.
- There exists a feature representation $ϕ$ such that $K(G,G') = ϕ(G)ϕ(G')$ which can
even be infinite-dimensional.

We could use a *bag-of-words (BOW)* for a graph. Recall that in NLP, BoW simply uses the
word count as features for documents with no ordering being considered. We regard nodes
as a word.

**Graph-Level Graphlet** features counts the number of different graphlets in a graph.
The graphlets here are not rooted and do not have to be connected. This definition of
graphlet is slightly different from the definition of node level features. A limitation of this
definition is that counting graphlets is expensive. Counting size-k gfor a raph with size n
by enumeration takes time $O(n^k)$ due to costly subgraph isomorphism tests. If a graph’s
node has bounded degree the time could be compressed down to $O(nd^{k−1})$.
so far we have only considered features related to the graph structure and not considered
the attributes of nodes and their neighbours.